In [1]:
import numpy as np
import scipy
import pandas as pd
from sklearn.metrics import classification_report
np.random.seed(1)

In [253]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Johnny
[nltk_data]     Peng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Train and Test set Split

In [2]:
data_full = pd.read_csv("data.csv")
X_train_df = data_full.sample(frac=0.8,random_state=200)

X_train = np.array(X_train_df['text'])
Y_train = np.array(X_train_df['label'])
print(len(X_train))

X_test = np.array(data_full.loc[~data_full.index.isin(X_train_df.index)]['text'])
Y_test = np.array(data_full.loc[~data_full.index.isin(X_train_df.index)]['label'])
print(len(Y_test))

t_size = int(len(X_train)*90/100)
print(len(X_train[:t_size]))

1623
406
1460


In [3]:
X_train_clean = []
X_test_clean = []

bad_chars = [';', ':', '!', "*","'", '[',']', '(', ')',',','?','’','-','|','"','$','•','\xa0'] 

for i in range(0,len(X_train)):
    temp = X_train[i].lower()
    for j in bad_chars:
        temp = temp.replace(j,'')
    X_train_clean.append(temp)

for i in range(0,len(X_test)):
    temp = X_test[i].lower()
    for j in bad_chars:
        temp = temp.replace(j,'')
    X_test_clean.append(temp)

# Text data
X_train_clean = np.array(X_train_clean)
X_test_clean = np.array(X_test_clean)

print ("X_train_clean shape: " + str(X_train_clean.shape))
print ("X_test_clean shape: " + str(X_test_clean.shape))

X_train_clean shape: (1623,)
X_test_clean shape: (406,)


In [4]:
X_full = np.hstack((X_train_clean,X_test_clean))
print(X_full.shape)
print(X_full[0])

(2029,)
stormfloodrainwaterhailorwind water damage due to excessive downpour the gutters couldnt cope with excessive rain and appears they overflowed under the eave. appears to just be wet ceiling in one room with only a few drops coming through into the room so at this stage just looks like minor plaster patching and painting required


In [5]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords

# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

# load doc and add to vocab
def add_doc_to_vocab(text, vocab):
    tokens = clean_doc(text)
    # update counts
    vocab.update(tokens)

# load all docs in a directory
def process_docs(docs, vocab):
    # walk through all files in the folder
    for i in range(0,len(docs)):
        add_doc_to_vocab(docs[i], vocab)

# define vocab
vocab = Counter()
# add all docs to vocab
process_docs(list(X_full), vocab)
# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

3998
[('stormfloodrainwaterhailorwind', 692), ('accidentallossordamage', 530), ('damage', 401), ('water', 331), ('insured', 317), ('electricmotorburnout', 195), ('damaged', 189), ('storm', 186), ('theftattemptedtheftorburglary', 163), ('escapeofliquidexcludingflood', 149), ('door', 146), ('house', 139), ('caused', 135), ('air', 126), ('home', 125), ('roof', 122), ('attached', 121), ('pump', 121), ('client', 121), ('ceiling', 119), ('back', 118), ('quote', 110), ('lost', 107), ('power', 100), ('due', 98), ('pool', 97), ('accidentalbreakage', 95), ('ring', 94), ('causing', 94), ('garage', 93), ('motor', 93), ('car', 88), ('tv', 88), ('claim', 87), ('phone', 86), ('also', 85), ('items', 85), ('working', 81), ('impact', 79), ('stolen', 78), ('fusion', 77), ('laptop', 77), ('carpet', 75), ('report', 73), ('accidentally', 72), ('room', 69), ('found', 69), ('electrical', 68), ('lightningorthunderbolt', 68), ('dropped', 67)]


In [439]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

2062


In [440]:
# save list to file
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w',encoding="utf-8")
    # write text
    file.write(data)
    # close file
    file.close()
 
# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

In [441]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r',encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

# load doc, clean and return line of tokens
def doc_to_line(text, vocab):
    # clean doc
    tokens = clean_doc(text)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

# load all docs in a directory
def process_docs(docs, vocab):
    lines = list()
    # walk through all files in the folder
    for i in range(0,len(docs)):
        line = doc_to_line(docs[i], vocab)
        # add to list
        lines.append(line)
    return lines

# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# load all training reviews
X_train_lines = process_docs(list(X_train_clean), vocab)
X_test_lines = process_docs(list(X_test_clean), vocab)
# summarize what we have
print(len(X_train_lines), len(X_test_lines))

1623 406


In [442]:
from keras.preprocessing.text import Tokenizer

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_train_lines)

# encode training data set
X_train_token = tokenizer.texts_to_matrix(X_train_lines, mode='count')
print(X_train_token.shape)

# encode training data set
X_test_token = tokenizer.texts_to_matrix(X_test_lines, mode='count')
print(X_test_token.shape)

(1623, 1998)
(406, 1998)


In [445]:
names = list(np.insert(np.array(list(tokenizer.word_index)),0,'0'))

# Classification

In [554]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion= 'gini', max_depth= 5, splitter= 'best',min_samples_leaf=10)
clf = clf.fit(X_train_token, Y_train)
predicts = clf.predict(X_train_token)
count = 0 

for i in range(0,Y_train.shape[0]):
    if predicts[i] == Y_train[i]:
        count = count + 1

count/Y_train.shape[0]

0.7597042513863216

In [555]:
count = 0

predicts_test = clf.predict(X_test_token)

for i in range(0,Y_test.shape[0]):
    if predicts_test[i] == Y_test[i]:
        count = count + 1

count/Y_test.shape[0]

0.7733990147783252

In [556]:
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

dot_data = tree.export_graphviz(clf, out_file=None, 
                      feature_names=names,  
                      class_names=list(('Building','Contents','Both')),  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.render("ClaimDesc") 

'ClaimDesc.pdf'

In [544]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# grid search
param_grid = [
 {'max_depth': [3,4,5],
 'criterion': ['entropy', 'gini'],
 'splitter': ['best', 'random'],
 'min_samples_leaf': [10,15,20,25,30]}
]
tree = GridSearchCV(DecisionTreeClassifier(), param_grid)
tree.fit(X_train_token, np.array(X_train_df['label']))
# Print grid search results
print('Grid search mean and stdev:\n')
for i in range(0,len(tree.cv_results_["params"])):
 print("{:0.3f} (+/-{:0.03f}) for {}".format(
 tree.cv_results_["mean_test_score"][i], tree.cv_results_["std_test_score"][i] *
2, tree.cv_results_["params"][i]))
# best params
print('\nBest parameters:', tree.best_params_)
# Evaluate on test set
print('\nClassification report ({}):\n'.format(key))
print(classification_report(Y_test, tree.predict(X_test_token)))

C:\Users\Johnny Peng\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Grid search mean and stdev:

0.734 (+/-0.023) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 10, 'splitter': 'best'}
0.731 (+/-0.024) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 10, 'splitter': 'random'}
0.734 (+/-0.023) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 15, 'splitter': 'best'}
0.734 (+/-0.023) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 15, 'splitter': 'random'}
0.732 (+/-0.025) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 20, 'splitter': 'best'}
0.732 (+/-0.025) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 20, 'splitter': 'random'}
0.732 (+/-0.025) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 25, 'splitter': 'best'}
0.732 (+/-0.025) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 25, 'splitter': 'random'}
0.732 (+/-0.025) for {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 30, 'splitter': 'best'}
0.732 (+/